In [34]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy
import os
import time as tm
from random import seed,randint

In [2]:
folder = 'D:\\Yolo\\facerec\\dataset\\'

In [7]:
def click_images(name):
    
    face_detect = cv2.CascadeClassifier('../facerec/haarcascade_frontalface_default.xml')
    img_folder = os.path.join(folder,name)
    
    if not os.path.exists(img_folder):
        os.mkdir(img_folder)
        os.chdir(img_folder)
    else:
        print('Folder exists...')
        os.chdir(img_folder)
        
    st = 3
    while st>=1:
        print('Taking picture in...',st)
        st -= 1
        tm.sleep(1)
    
    cam = cv2.VideoCapture(0)
    pic = 0
    try:
        while pic<50:
            pic += 1
            _,img = cam.read()
            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            face = face_detect.detectMultiScale(img,scaleFactor=1.1,minNeighbors=7,minSize=(40,40))
            for (x,y,w,h) in face:
                img = img[y:y+h,x:x+w]
            img = cv2.resize(img,(50,50))
            cv2.imwrite(os.path.join(img_folder,f'{pic}.jpg'),img)
            print(pic,':clicked!')
    except Exception as e:
        print("Error occured:",e)
        cam.release()
        exit(0)
    
    cam.release()

In [8]:
def getData():
    data = image_dataset_from_directory(folder)
    return data

In [9]:
name = input('Enter name')
click_images(name)

Taking picture in... 3
Taking picture in... 2
Taking picture in... 1
1 :clicked!
2 :clicked!
3 :clicked!
4 :clicked!
5 :clicked!
6 :clicked!
7 :clicked!
8 :clicked!
9 :clicked!
10 :clicked!
11 :clicked!
12 :clicked!
13 :clicked!
14 :clicked!
15 :clicked!
16 :clicked!
17 :clicked!
18 :clicked!
19 :clicked!
20 :clicked!
21 :clicked!
22 :clicked!
23 :clicked!
24 :clicked!
25 :clicked!
26 :clicked!
27 :clicked!
28 :clicked!
29 :clicked!
30 :clicked!
31 :clicked!
32 :clicked!
33 :clicked!
34 :clicked!
35 :clicked!
36 :clicked!
37 :clicked!
38 :clicked!
39 :clicked!
40 :clicked!
41 :clicked!
42 :clicked!
43 :clicked!
44 :clicked!
45 :clicked!
46 :clicked!
47 :clicked!
48 :clicked!
49 :clicked!
50 :clicked!


In [30]:
data = getData()
# print(len(data))
data = data.map(lambda x,y: (x/255,y))
data_iter = data.as_numpy_iterator()
batch = data_iter.next()

Found 100 files belonging to 2 classes.


(32, 256, 256, 3)

In [26]:
train_size = int(len(data)*.5)
val_size = int(len(data)*.25)
test_size = int(len(data)*.25)

In [27]:
train = data.take(train_size)
valid = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [32]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 246016)            0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [33]:
results = model.fit(train,epochs=10,validation_data=valid)

Epoch 1/10
2/2 [==============================] - 44s 22s/step - loss: 1.9735 - accuracy: 0.5000 - val_loss: 0.8161 - val_accuracy: 0.5312
Epoch 2/10
2/2 [==============================] - 35s 24s/step - loss: 0.8073 - accuracy: 0.4688 - val_loss: 0.6644 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 24s 9s/step - loss: 0.6595 - accuracy: 0.5312 - val_loss: 0.5456 - val_accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 12s 7s/step - loss: 0.5368 - accuracy: 0.8281 - val_loss: 0.4859 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 13s 7s/step - loss: 0.4617 - accuracy: 0.9375 - val_loss: 0.3472 - val_accuracy: 0.9688
Epoch 6/10
2/2 [==============================] - 13s 7s/step - loss: 0.3206 - accuracy: 1.0000 - val_loss: 0.2313 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 12s 6s/step - loss: 0.1978 - accuracy: 0.9844 - val_loss: 0.1070 - val_accuracy: 1.0000
Epoch 8/10
2/2 [================

In [35]:
precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()

In [36]:
for i in test:
    X,y = i
    pred = model.predict(X)
    precision.update_state(y,pred)
    recall.update_state(y,pred)
    accuracy.update_state(y,pred)
    
print('precision:{}\nrecall:{}\naccuracy:{}'.format(precision.result(),recall.result(),accuracy.result()))

1/1 [==============================] - 1s 1s/step
precision:1.0
recall:1.0
accuracy:1.0


In [68]:
test_fol = r'D:\Yolo\facerec\test_imgs'
timg = cv2.imread(os.path.join(test_fol,'5.jpg'))
timg = tf.image.resize(timg,(256,256))
print(timg.shape)

(256, 256, 3)


In [70]:
preds = model.predict(np.expand_dims(timg/255,0))
print(preds)
if preds[0][0]>0.5:
    print('User not found...')
else:
    print('User found!')

1/1 [==============================] - 0s 160ms/step
[[0.178781]]
User found!
